# Clean up for womens-march-madness.csv

## Loading CSV

Load the necessary libraries for the clean-up, as well as the provided dataset

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
data = pd.read_csv("../data/raw/womens-march-madness/womens-march-madness.csv")
data.head()

,Year,School,Seed,Conference,Conf. W,Conf. L,Conf. %,Conf. place,Reg. W,Reg. L,Reg. %,How qual,1st game at home?,Tourney W,Tourney L,Tourney finish,Full W,Full L,Full %
0,1982,Arizona St.,4,Western Collegiate,-,-,-,-,23,6,79.3,at-large,Y,1,1,RSF,24,7,77.4
1,1982,Auburn,7,Southeastern,-,-,-,-,24,4,85.7,at-large,N,0,1,1st,24,5,82.8
2,1982,Cheyney,2,Independent,-,-,-,-,24,2,92.3,at-large,Y,4,1,N2nd,28,3,90.3
3,1982,Clemson,5,Atlantic Coast,6,3,66.7,4th,20,11,64.5,at-large,N,0,1,1st,20,12,62.5
4,1982,Drake,4,Missouri Valley,-,-,-,-,26,6,81.3,auto,Y,2,1,RF,28,7,80


## Renaming columns
Changing the column names to a more interpretable version:

In [3]:
column_names = {
    'Year' : 'year',
    'School' : 'school',
    'Seed' : 'seed',
    'Conference' : 'conference',
    'Conf. W' : 'conf_wins',
    'Conf. L' : 'conf_losses',
    'Conf. %' : 'conf_wins_pct',
    'Conf. place' : 'conf_place',
    'Reg. W' : 'reg_wins',
    'Reg. L' : 'reg_losses',
    'Reg. %' : 'reg_wins_pct',
    'How qual' : 'bid',
    '1st game at home?' : 'first_game_at_home',
    'Tourney W' : 'tourney_wins',
    'Tourney L' : 'tourney_losses',
    'Tourney finish' : 'tourney_finish',
    'Full W' : 'total_wins',
    'Full L' : 'total_losses',
    'Full %' : 'total_wins_pct'
}
data = data.rename(columns = column_names)
data.head()

,year,school,seed,conference,conf_wins,conf_losses,conf_wins_pct,conf_place,reg_wins,reg_losses,reg_wins_pct,bid,first_game_at_home,tourney_wins,tourney_losses,tourney_finish,total_wins,total_losses,total_wins_pct
0,1982,Arizona St.,4,Western Collegiate,-,-,-,-,23,6,79.3,at-large,Y,1,1,RSF,24,7,77.4
1,1982,Auburn,7,Southeastern,-,-,-,-,24,4,85.7,at-large,N,0,1,1st,24,5,82.8
2,1982,Cheyney,2,Independent,-,-,-,-,24,2,92.3,at-large,Y,4,1,N2nd,28,3,90.3
3,1982,Clemson,5,Atlantic Coast,6,3,66.7,4th,20,11,64.5,at-large,N,0,1,1st,20,12,62.5
4,1982,Drake,4,Missouri Valley,-,-,-,-,26,6,81.3,auto,Y,2,1,RF,28,7,80


## Column Analysis and Reformatting

Analyzing and replacing the contents of the 'Conf. place' : 'conf_place' column:

In [4]:
data['conf_place'].unique()

array(['-', '4th', '1st', '2nd', '3rd', 'T1st W', '3rd E', '2nd E',
       'T2nd', '1st E', 'T2nd W', 'T1st E', 'T1st', 'T1st N', 'T3rd',
       '1st W', '2nd W', '1st S', '2nd S', 'T4th', 'T2nd E', 'T6th',
       '5th', '6th', 'T5th', 'T8th', 'T7th', '7th', '1st-B', '1st-W',
       '2nd-E', '2nd-6', '1st-E', '1st-R', 'T2nd-W', '1st-6', 'T1st-B',
       'T1st-W', '1st-P', '1st-M', 'T1st-M', '2nd-A', '1st-A', '1st-N',
       '2nd-7', 'T3rd-M', 'T2nd-6', '1st-7', 'T2nd-P', '2nd-N', '2nd-W',
       '8th', 'T1st-A', '2nd-M', '3rd-W', '4th-N', '9th', 'T1st-E',
       '10th', 'T9th', '1st Div.', '3rd Div', 'T3rd Div.'], dtype=object)

- Numerical Places:

'1st', '2nd', '3rd', etc. — clearly ranked within the conference.

- Tied Places:

'T1st', 'T2nd', etc. — team tied for that position.

'T1st-W', 'T2nd-E', etc. — tied for a position within a division (e.g., West, East).

- Divisions within Conferences:

'1st W', '2nd E', '3rd-W', etc. — first, second, or third place in a regional division (West, East, etc.).

- Ambiguous Codes:

'1st-6', 'T2nd-6', '2nd-7' — unknown for divisions (e.g., "Group 6" or region 6).

'1st-B', '1st-R', '1st-P', '1st-M', etc. — probably shorthand for colored or named divisions (e.g., Blue, Red, Pacific, Mountain).

'1st Div.', '3rd Div', 'T3rd Div.' — older way of denoting division standing.

- '-' — missing or not applicable (e.g., for Independent schools not in a conference).

Functions to replace 'conf_place' with two columns 'conf_rank' and 'division'

In [5]:
def extract_conf_rank(conf_place):
    if pd.isna(conf_place) or conf_place == '-':
        return np.nan
    # Match patterns like '1st', 'T2nd', '3rd-W', 'T1st-N', etc.
    match = re.search(r'(\d+)(st|nd|rd|th)', conf_place)
    if match:
        return int(match.group(1))
    return np.nan

def extract_division(conf_place):
    if pd.isna(conf_place) or conf_place == '-':
        return np.nan
    # Look for division info after rank (like 'W', 'E', '6', etc.)
    match = re.search(r'(?:\d+(?:st|nd|rd|th)[\s-]?)([A-Za-z0-9]+)?', conf_place)
    if match:
        return match.group(1)
    return np.nan

In [6]:
data['conf_rank'] = data['conf_place'].apply(extract_conf_rank)
data['division'] = data['conf_place'].apply(extract_division)
data.head()

,year,school,seed,conference,conf_wins,conf_losses,conf_wins_pct,conf_place,reg_wins,reg_losses,...,bid,first_game_at_home,tourney_wins,tourney_losses,tourney_finish,total_wins,total_losses,total_wins_pct,conf_rank,division
0,1982,Arizona St.,4,Western Collegiate,-,-,-,-,23,6,...,at-large,Y,1,1,RSF,24,7,77.4,NaN,NaN
1,1982,Auburn,7,Southeastern,-,-,-,-,24,4,...,at-large,N,0,1,1st,24,5,82.8,NaN,NaN
2,1982,Cheyney,2,Independent,-,-,-,-,24,2,...,at-large,Y,4,1,N2nd,28,3,90.3,NaN,NaN
3,1982,Clemson,5,Atlantic Coast,6,3,66.7,4th,20,11,...,at-large,N,0,1,1st,20,12,62.5,4.0,None
4,1982,Drake,4,Missouri Valley,-,-,-,-,26,6,...,auto,Y,2,1,RF,28,7,80,NaN,NaN


Analyzing and replacing the contents of the 'Tourney finish' : 'tourney_finish' column:

According to the documentation:

The round of the final game for each team. OR=opening-round loss (1983 only); 1st=first-round loss; 2nd=second-round loss; RSF=loss in the Sweet 16; RF=loss in the Elite Eight; NSF=loss in the national semifinals; N2nd=national runner-up; Champ=national champions.

We'll recode the content of the column to more interpretable names.

In [7]:
data['tourney_finish'].unique()

array(['RSF', '1st', 'N2nd', 'RF', 'Champ', 'NSF', 'OR', '2nd'],
      dtype=object)

In [8]:
finish_map = {
    'OR': 'opening_round_loss',
    '1st': 'first_round_loss',
    '2nd': 'second_round_loss',
    'RSF': 'top_16_loss',
    'RF': 'top_8_loss',
    'NSF': 'top_4_loss',
    'N2nd': 'top_2_loss',
    'Champ': 'champ'
}

data['tourney_finish'] = data['tourney_finish'].replace(finish_map)

data.head()

,year,school,seed,conference,conf_wins,conf_losses,conf_wins_pct,conf_place,reg_wins,reg_losses,...,bid,first_game_at_home,tourney_wins,tourney_losses,tourney_finish,total_wins,total_losses,total_wins_pct,conf_rank,division
0,1982,Arizona St.,4,Western Collegiate,-,-,-,-,23,6,...,at-large,Y,1,1,top_16_loss,24,7,77.4,NaN,NaN
1,1982,Auburn,7,Southeastern,-,-,-,-,24,4,...,at-large,N,0,1,first_round_loss,24,5,82.8,NaN,NaN
2,1982,Cheyney,2,Independent,-,-,-,-,24,2,...,at-large,Y,4,1,top_2_loss,28,3,90.3,NaN,NaN
3,1982,Clemson,5,Atlantic Coast,6,3,66.7,4th,20,11,...,at-large,N,0,1,first_round_loss,20,12,62.5,4.0,None
4,1982,Drake,4,Missouri Valley,-,-,-,-,26,6,...,auto,Y,2,1,top_8_loss,28,7,80,NaN,NaN


We're going to make this column an ordered categorical.

In [9]:
finish_order = [
    'opening_round_loss',
    'first_round_loss',
    'second_round_loss',
    'top_16_loss',
    'top_8_loss',
    'top_4_loss',
    'top_2_loss',
    'champ'
]

data['tourney_finish'] = pd.Categorical(
    data['tourney_finish'], 
    categories=finish_order, 
    ordered=True
)

data.head()

,year,school,seed,conference,conf_wins,conf_losses,conf_wins_pct,conf_place,reg_wins,reg_losses,...,bid,first_game_at_home,tourney_wins,tourney_losses,tourney_finish,total_wins,total_losses,total_wins_pct,conf_rank,division
0,1982,Arizona St.,4,Western Collegiate,-,-,-,-,23,6,...,at-large,Y,1,1,top_16_loss,24,7,77.4,NaN,NaN
1,1982,Auburn,7,Southeastern,-,-,-,-,24,4,...,at-large,N,0,1,first_round_loss,24,5,82.8,NaN,NaN
2,1982,Cheyney,2,Independent,-,-,-,-,24,2,...,at-large,Y,4,1,top_2_loss,28,3,90.3,NaN,NaN
3,1982,Clemson,5,Atlantic Coast,6,3,66.7,4th,20,11,...,at-large,N,0,1,first_round_loss,20,12,62.5,4.0,None
4,1982,Drake,4,Missouri Valley,-,-,-,-,26,6,...,auto,Y,2,1,top_8_loss,28,7,80,NaN,NaN


Analyzing and replacing the contents of the 'Seed' : 'seed' column:

In [10]:
data.loc[data['seed'] == '(OR)', 'seed'] = '0'

## Changing datatypes:

In [11]:
data.replace("-", np.nan, inplace=True)

data['seed'] = data['seed'].astype('int')
data['conf_wins'] = data['conf_wins'].astype('float')
data['conf_losses'] = data['conf_losses'].astype('float')
data['conf_wins_pct'] = data['conf_wins_pct'].astype('float')
data['total_wins_pct'] = data['total_wins_pct'].str.replace('\\', '')
data['first_game_at_home'] = data['first_game_at_home'].str.replace('^', '')
data['total_wins_pct'] = data['total_wins_pct'].astype('float')

data.head()

,year,school,seed,conference,conf_wins,conf_losses,conf_wins_pct,conf_place,reg_wins,reg_losses,...,bid,first_game_at_home,tourney_wins,tourney_losses,tourney_finish,total_wins,total_losses,total_wins_pct,conf_rank,division
0,1982,Arizona St.,4,Western Collegiate,NaN,NaN,NaN,NaN,23,6,...,at-large,Y,1,1,top_16_loss,24,7,77.4,NaN,NaN
1,1982,Auburn,7,Southeastern,NaN,NaN,NaN,NaN,24,4,...,at-large,N,0,1,first_round_loss,24,5,82.8,NaN,NaN
2,1982,Cheyney,2,Independent,NaN,NaN,NaN,NaN,24,2,...,at-large,Y,4,1,top_2_loss,28,3,90.3,NaN,NaN
3,1982,Clemson,5,Atlantic Coast,6.0,3.0,66.7,4th,20,11,...,at-large,N,0,1,first_round_loss,20,12,62.5,4.0,None
4,1982,Drake,4,Missouri Valley,NaN,NaN,NaN,NaN,26,6,...,auto,Y,2,1,top_8_loss,28,7,80.0,NaN,NaN


## NaN Standardization

In [12]:
data.replace(to_replace=[pd.NA, "nan", "NaN", "None", None], value=np.nan, inplace=True)

## Results Review

In [13]:
for column in data.columns:
    print(f"Column: {column}")
    print(f"Data type: {data[column].dtype}")
    print(f"Unique values: {data[column].unique()}\n")

Column: year
Data type: int64
Unique values: [1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995
 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012 2013 2014 2015 2016 2017 2018]

Column: school
Data type: object
Unique values: ['Arizona St.' 'Auburn' 'Cheyney' 'Clemson' 'Drake' 'East Carolina'
 'Georgia' 'Howard' 'Illinois' 'Jackson St.' 'Kansas St.' 'Kent St.'
 'Kentucky' 'Long Beach St.' 'Louisiana Tech' 'Maryland' 'Memphis'
 'Missouri' 'NC State' 'Northwestern' 'Ohio St.' 'Old Dominion' 'Ole Miss'
 'Oregon' 'Penn St.' "Saint Peter's" 'SFA' 'South Carolina'
 'Southern California' 'Stanford' 'Tennessee' 'Tennessee Tech'
 'Central Mich.' 'Dartmouth' 'Florida St.' 'Illinois St.' 'Indiana'
 'La Salle' 'La.-Monroe' 'Louisville' 'Middle Tenn.' 'Monmouth' 'Montana'
 'North Carolina' 'Oregon St.' 'South Carolina St.' "St. John's (NY)"
 'Texas' 'UCLA' 'Utah' 'Alabama' 'BYU' 'LSU' 'San Diego St.' 'Texas Tech'
 'UNLV' 'Virginia' 'Holy Cros

## Final Reorganization of Columns

In [14]:
data.columns

Index(['year', 'school', 'seed', 'conference', 'conf_wins', 'conf_losses',
       'conf_wins_pct', 'conf_place', 'reg_wins', 'reg_losses', 'reg_wins_pct',
       'bid', 'first_game_at_home', 'tourney_wins', 'tourney_losses',
       'tourney_finish', 'total_wins', 'total_losses', 'total_wins_pct',
       'conf_rank', 'division'],
      dtype='object')

In [15]:
data = data[['year', 'school', 'seed', 'conference', 'conf_wins', 'conf_losses',
       'conf_wins_pct', 'conf_rank', 'division', 'reg_wins', 'reg_losses', 'reg_wins_pct',
       'bid', 'first_game_at_home', 'tourney_wins', 'tourney_losses',
       'tourney_finish', 'total_wins', 'total_losses', 'total_wins_pct'
       ]]

data.head()

,year,school,seed,conference,conf_wins,conf_losses,conf_wins_pct,conf_rank,division,reg_wins,reg_losses,reg_wins_pct,bid,first_game_at_home,tourney_wins,tourney_losses,tourney_finish,total_wins,total_losses,total_wins_pct
0,1982,Arizona St.,4,Western Collegiate,NaN,NaN,NaN,NaN,NaN,23,6,79.3,at-large,Y,1,1,top_16_loss,24,7,77.4
1,1982,Auburn,7,Southeastern,NaN,NaN,NaN,NaN,NaN,24,4,85.7,at-large,N,0,1,first_round_loss,24,5,82.8
2,1982,Cheyney,2,Independent,NaN,NaN,NaN,NaN,NaN,24,2,92.3,at-large,Y,4,1,top_2_loss,28,3,90.3
3,1982,Clemson,5,Atlantic Coast,6.0,3.0,66.7,4.0,NaN,20,11,64.5,at-large,N,0,1,first_round_loss,20,12,62.5
4,1982,Drake,4,Missouri Valley,NaN,NaN,NaN,NaN,NaN,26,6,81.3,auto,Y,2,1,top_8_loss,28,7,80.0


## Saving CSV

In [17]:
data.to_csv('../data/clean/womensmarchmadness.csv', index=False)

## Validating

In [19]:
clean = pd.read_csv("../data/clean/womensmarchmadness.csv")

clean.head()

,year,school,seed,conference,conf_wins,conf_losses,conf_wins_pct,conf_rank,division,reg_wins,reg_losses,reg_wins_pct,bid,first_game_at_home,tourney_wins,tourney_losses,tourney_finish,total_wins,total_losses,total_wins_pct
0,1982,Arizona St.,4,Western Collegiate,NaN,NaN,NaN,NaN,NaN,23,6,79.3,at-large,Y,1,1,top_16_loss,24,7,77.4
1,1982,Auburn,7,Southeastern,NaN,NaN,NaN,NaN,NaN,24,4,85.7,at-large,N,0,1,first_round_loss,24,5,82.8
2,1982,Cheyney,2,Independent,NaN,NaN,NaN,NaN,NaN,24,2,92.3,at-large,Y,4,1,top_2_loss,28,3,90.3
3,1982,Clemson,5,Atlantic Coast,6.0,3.0,66.7,4.0,NaN,20,11,64.5,at-large,N,0,1,first_round_loss,20,12,62.5
4,1982,Drake,4,Missouri Valley,NaN,NaN,NaN,NaN,NaN,26,6,81.3,auto,Y,2,1,top_8_loss,28,7,80.0


In [20]:
clean.shape

(2092, 20)